<a href="https://colab.research.google.com/github/NadiaCopello/Spatial_Analysis_of_Peru_Protected_Areas/blob/main/PREGUNTA_2_Y_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install requests rasterio numpy


In [ ]:
import os
import requests
from datetime import datetime

# Credenciales del Earthdata
USERNAME = 'nadiajc'
PASSWORD = 'zz4j^z2Sj-FUrwn'

# Establezco un directorio de salida
output_dir = 'MODIS_MOD44B_2024'
os.makedirs(output_dir, exist_ok=True)

# Averigué que tiles son los de Perú
tiles = ['h09v09', 'h09v10', 'h10v08', 'h10v09', 'h10v10', 'h11v09', 'h11v10']

# Pongo el año que me piden, es decir 2024.
year = 2024
doy = 1  # Día 1 del año
date_str = f'{year}{doy:03d}'

# URL base
base_url = f'https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/61/MOD44B/2024/'

# Sesión con autenticación
session = requests.Session()
session.auth = (USERNAME, PASSWORD)

for tile in tiles:
    filename = f'MOD44B.A{date_str}.{tile}.061.*.hdf'
    response = session.get(base_url)
    if response.status_code == 200:
        # Buscar el archivo que coincide con el tile
        for line in response.text.split('\n'):
            if tile in line and '.hdf' in line:
                start = line.find('MOD44B')
                end = line.find('.hdf') + 4
                hdf_filename = line[start:end]
                file_url = base_url + hdf_filename
                output_path = os.path.join(output_dir, hdf_filename)
                if not os.path.exists(output_path):
                    print(f'Descargando {hdf_filename}...')
                    r = session.get(file_url, stream=True)
                    with open(output_path, 'wb') as f:
                        for chunk in r.iter_content(chunk_size=8192):
                            f.write(chunk)
    else:
        print(f'Error al acceder a {base_url}')


Una vez descargados los archivos HDF, necesito extraer el subdataset Percent_Tree_Cover y reproyectarlo al sistema de coordenadas EPSG:4326.

In [ ]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np

# Lista de archivos HDF descargados
hdf_files = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith('.hdf')]

# Directorio para los archivos TIFF reproyectados
tiff_dir = 'MODIS_TIFF'
os.makedirs(tiff_dir, exist_ok=True)

for hdf_file in hdf_files:
    with rasterio.open(f'HDF4_EOS:EOS_GRID:"{hdf_file}":MOD_Grid_Tree_Cover:Percent_Tree_Cover') as src:
        # Reproyectar a EPSG:4326
        dst_crs = 'EPSG:4326'
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        tile_name = os.path.basename(hdf_file).replace('.hdf', '.tif')
        output_tiff = os.path.join(tiff_dir, tile_name)

        with rasterio.open(output_tiff, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)


In [ ]:
from rasterio.merge import merge
import glob

# Lista de archivos TIFF reproyectados
tiff_files = glob.glob(os.path.join(tiff_dir, '*.tif'))

# Abro los archivos
src_files_to_mosaic = []
for fp in tiff_files:
    src = rasterio.open(fp)
    src_files_to_mosaic.append(src)

# Creo el mosaico
mosaic, out_trans = merge(src_files_to_mosaic)

# Actualizo los metadatos
out_meta = src.meta.copy()
out_meta.update({
    'height': mosaic.shape[1],
    'width': mosaic.shape[2],
    'transform': out_trans,
    'crs': 'EPSG:4326'
})

# Para guardar el mosaico
mosaic_path = 'vcf_2024_peru.tif'
with rasterio.open(mosaic_path, 'w', **out_meta) as dest:
    dest.write(mosaic)


IndexError: list index out of range

pregunta 3


In [ ]:
# Análisis de Zonas de Amortiguamiento para Áreas Protegidas
# Análisis de efectos de borde en cobertura forestal

import geopandas as gpd
import rasterio
import numpy as np
import pandas as pd
from rasterio.mask import mask
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')


# CARGAR DATOS

def cargar_datos():
    """
    Carga los datos necesarios para el análisis
    """
    print(" Cargando datos")

    # Cargar shapefile de las áreas protegidas
    # voy a suponer que tengo un archivo shapefile con las áreas protegidas
    try:
        areas_protegidas = gpd.read_file("areas_protegidas.shp")
        print(f" Áreas protegidas cargadas: {len(areas_protegidas)} polígonos")
    except:
        print(" No se encontró el archivo. Creando datos de ejemplo...")
        # Creo datos de ejemplo si no existe el archivo
        areas_protegidas = crear_datos_ejemplo()

    # Cargar ráster de cobertura forestal
    # también asumo que un archivo raster con porcentaje de cobertura arbórea
    try:
        with rasterio.open("cobertura_forestal.tif") as src:
            forest_data = src.read(1)
            forest_profile = src.profile
        print(" Ráster de cobertura forestal cargado")
    except:
        print(" No se encontró el ráster. Creando datos simulados...")
        forest_data, forest_profile = crear_raster_ejemplo()

    return areas_protegidas, forest_data, forest_profile

def crear_datos_ejemplo():
    """
    Crea datos de ejemplo para demostración
    """
    # Creo los polígonos de ejemplo (áreas protegidas ficticias)
    polygons = []
    for i in range(5):
        # Coordenadas aleatorias (ajusta según tu región de interés)
        center_lon = -75.0 + i * 0.1
        center_lat = -10.0 + i * 0.1

        # Creo polígono cuadrado de ~5km x 5km
        size = 0.045  # aprox 5km en grados
        coords = [
            (center_lon - size, center_lat - size),
            (center_lon + size, center_lat - size),
            (center_lon + size, center_lat + size),
            (center_lon - size, center_lat + size),
            (center_lon - size, center_lat - size)
        ]

        polygons.append({
            'id_parque': f'PA_{i+1}',
            'nombre': f'Área Protegida {i+1}',
            'geometry': Polygon(coords)
        })

    return gpd.GeoDataFrame(polygons, crs='EPSG:4326')

def crear_raster_ejemplo():
    """
    Crea un ráster de ejemplo con datos simulados de cobertura forestal
    """
    # Creo una grilla de ejemplo
    height, width = 1000, 1000

    # Simulo la cobertura forestal (valores entre 0-100%)
    np.random.seed(42)
    forest_cover = np.random.beta(2, 2, (height, width)) * 100

    # Pongo la configuración del ráster
    profile = {
        'driver': 'GTiff',
        'dtype': 'float32',
        'nodata': -9999,
        'width': width,
        'height': height,
        'count': 1,
        'crs': 'EPSG:4326',
        'transform': rasterio.transform.from_bounds(
            -76, -11, -74, -9, width, height
        )
    }

    return forest_cover.astype('float32'), profile


# GENERAR ZONAS DE AMORTIGUAMIENTO

def crear_buffers(geometria, ancho_km):
    """
    Crea buffers interno y externo para una geometría dada

    Args:
        geometria: Polígono de área protegida
        ancho_km: Ancho del buffer en kilómetros

    Returns:
        buffer_interno, buffer_externo: Geometrías de los buffers
    """
    # Converto km a grados (aproximación: 1 grado ≈ 111 km)
    ancho_grados = ancho_km / 111.0

    # Y Buffer negativo (interno) - erosión
    buffer_interno = geometria.buffer(-ancho_grados)

    # Y  Buffer positivo (externo) - dilatación
    buffer_total = geometria.buffer(ancho_grados)

    # Y Buffer exterior = buffer total - geometría original
    buffer_externo = buffer_total.difference(geometria)

    return buffer_interno, buffer_externo

def procesar_todos_buffers(areas_protegidas, anchos_km):
    """
    Procesa todos los buffers para todas las áreas protegidas

    Args:
        areas_protegidas: GeoDataFrame con áreas protegidas
        anchos_km: Lista de anchos de buffer en km

    Returns:
        DataFrame con todas las combinaciones de buffers
    """
    print("\n Generando zonas de amortiguamiento...")

    resultados = []

    for idx, area in areas_protegidas.iterrows():
        id_parque = area['id_parque']
        geometria = area['geometry']

        print(f"Procesando {id_parque}...")

        for ancho in anchos_km:
            try:
                buffer_interno, buffer_externo = crear_buffers(geometria, ancho)

                # Es importante verificar que los buffers son válidos
                if buffer_interno.is_empty or buffer_externo.is_empty:
                    print(f"Buffer vacío para {id_parque} con {ancho}km")
                    continue

                # Agrego buffer interno
                resultados.append({
                    'id_parque': id_parque,
                    'buffer_km': ancho,
                    'dentro_fuera': 'interno',
                    'geometry': buffer_interno
                })

                # Agrego buffer externo
                resultados.append({
                    'id_parque': id_parque,
                    'buffer_km': ancho,
                    'dentro_fuera': 'externo',
                    'geometry': buffer_externo
                })

            except Exception as e:
                print(f" Error procesando {id_parque} con {ancho}km: {e}")
                continue

    return gpd.GeoDataFrame(resultados, crs=areas_protegidas.crs)



In [ ]:

# AHORA EXTRAER ESTADÍSTICAS ZONALES

def extraer_estadisticas_zonales(geometria, raster_data, raster_profile):
    """
    Extrae estadísticas de un ráster dentro de una geometría

    Args:
        geometria: Geometría para recortar el ráster
        raster_data: Array del ráster
        raster_profile: Metadatos del ráster

    Returns:
        dict con estadísticas (media, mediana, std, n_pixeles)
    """
    try:
        # Creo un dataset temporal en memoria
        with rasterio.MemoryFile() as memfile:
            with memfile.open(**raster_profile) as dataset:
                dataset.write(raster_data, 1)

                # Recorto el ráster con la geometría
                masked_data, _ = mask(dataset, [geometria], crop=True, nodata=-9999)
                masked_data = masked_data[0]  # Primera banda

                # Elimino valores NoData
                valid_data = masked_data[masked_data != -9999]

                if len(valid_data) == 0:
                    return {
                        'media_tc': np.nan,
                        'mediana_tc': np.nan,
                        'sd_tc': np.nan,
                        'n_pixeles': 0
                    }

                return {
                    'media_tc': float(np.mean(valid_data)),
                    'mediana_tc': float(np.median(valid_data)),
                    'sd_tc': float(np.std(valid_data)),
                    'n_pixeles': len(valid_data)
                }

    except Exception as e:
        print(f"Error en estadísticas zonales: {e}")
        return {
            'media_tc': np.nan,
            'mediana_tc': np.nan,
            'sd_tc': np.nan,
            'n_pixeles': 0
        }

def procesar_estadisticas_todas_zonas(buffers_gdf, raster_data, raster_profile):
    """
    Procesa estadísticas zonales para todas las zonas de buffer
    """
    print("\n Extrayendo estadísticas zonales...")

    resultados = []

    for idx, row in buffers_gdf.iterrows():
        print(f"Procesando {row['id_parque']} - {row['buffer_km']}km - {row['dentro_fuera']}")

        stats = extraer_estadisticas_zonales(
            row['geometry'], raster_data, raster_profile
        )

        resultado = {
            'id_parque': row['id_parque'],
            'buffer_km': row['buffer_km'],
            'dentro_fuera': row['dentro_fuera'],
            **stats
        }

        resultados.append(resultado)

    return pd.DataFrame(resultados)

# LUEGO SE HARÁ EL ANÁLISIS DE REGRESIÓN

def preparar_datos_regresion(df_estadisticas):
    """
    Prepara los datos para la regresión binaria
    """
    print("\n Preparando datos para regresión...")

    # CreO variable de tratamiento D (1 = interno/dentro, 0 = externo/fuera)
    df_regresion = df_estadisticas.copy()
    df_regresion['D'] = (df_regresion['dentro_fuera'] == 'interno').astype(int)

    # EliminO filas con valores faltantes
    df_regresion = df_regresion.dropna(subset=['media_tc', 'n_pixeles'])
    df_regresion = df_regresion[df_regresion['n_pixeles'] > 0]

    print(f"Datos para regresión: {len(df_regresion)} observaciones")

    return df_regresion

def ejecutar_regresion(df_regresion):
    """
    Ejecuta la regresión binaria ponderada
    """
    print("\n Ejecutando regresión...")

    # Las Variables
    X = df_regresion[['D']].values
    y = df_regresion['media_tc'].values
    pesos = df_regresion['n_pixeles'].values

    # Regresión lineal ponderada
    modelo = LinearRegression()
    modelo.fit(X, y, sample_weight=pesos)

    # Predicciones
    y_pred = modelo.predict(X)

    # Métricas
    r2 = r2_score(y, y_pred, sample_weight=pesos)

    # Resultados
    intercepto = modelo.intercept_
    coef_D = modelo.coef_[0]

    print(f"Intercepto (β₀): {intercepto:.4f}")
    print(f"Coeficiente D (β₁): {coef_D:.4f}")
    print(f"R² ponderado: {r2:.4f}")

    # Interpretación
    print(f"\n Interpretación:")
    print(f"• Cobertura forestal promedio FUERA de áreas protegidas: {intercepto:.2f}%")
    print(f"• Diferencia DENTRO vs FUERA: {coef_D:.2f} puntos porcentuales")

    if coef_D > 0:
        print(f"• Las áreas protegidas tienen MAYOR cobertura forestal (+{coef_D:.2f}%)")
    else:
        print(f"• Las áreas protegidas tienen MENOR cobertura forestal ({coef_D:.2f}%)")

    return {
        'modelo': modelo,
        'intercepto': intercepto,
        'coef_D': coef_D,
        'r2': r2,
        'n_obs': len(df_regresion)
    }


# LA FUNCIÓN PRINCIPAL
def analisis_completo():
    """
    Ejecuta el análisis completo de zonas de amortiguamiento
    """
    print(" INICIANDO ANÁLISIS DE ZONAS DE AMORTIGUAMIENTO")
    print("=" * 60)

    # 1. CargO datos
    areas_protegidas, forest_data, forest_profile = cargar_datos()

    # 2. Defino anchos de buffer
    anchos_km = [5, 10, 20, 25]

    # 3. Genero los buffers
    buffers_gdf = procesar_todos_buffers(areas_protegidas, anchos_km)
    print(f" Generados {len(buffers_gdf)} buffers")

    # 4. Extraigo  estadísticas zonales
    df_estadisticas = procesar_estadisticas_todas_zonas(
        buffers_gdf, forest_data, forest_profile
    )

    # 5. Guardo resultados en CSV
    archivo_csv = "resultados_buffers.csv"
    df_estadisticas.to_csv(archivo_csv, index=False)
    print(f"✅ Resultados guardados en: {archivo_csv}")

    # 6. Muestro la muestra de resultados...valga la redundancia
    print(f"\n Muestra de resultados:")
    print(df_estadisticas.head(10).to_string())

    # 7. Análisis de regresión
    df_regresion = preparar_datos_regresion(df_estadisticas)
    resultados_regresion = ejecutar_regresion(df_regresion)

    # 8. Visualización
    crear_visualizaciones(df_estadisticas, df_regresion)

    print("\n ANÁLISIS COMPLETADO")
    return df_estadisticas, resultados_regresion

def crear_visualizaciones(df_estadisticas, df_regresion):
    """
    Crea visualizaciones de los resultados
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))

    # Gráfico 1: Cobertura promedio por buffer y tipo
    ax1 = axes[0, 0]
    df_pivot = df_estadisticas.pivot_table(
        values='media_tc',
        index='buffer_km',
        columns='dentro_fuera',
        aggfunc='mean'
    )
    df_pivot.plot(kind='bar', ax=ax1)
    ax1.set_title('Cobertura Forestal Promedio por Zona')
    ax1.set_ylabel('% Cobertura Arbórea')
    ax1.legend(title='Zona')

    # Gráfico 2: Distribución de cobertura
    ax2 = axes[0, 1]
    df_estadisticas[df_estadisticas['dentro_fuera'] == 'interno']['media_tc'].hist(
        alpha=0.7, label='Interno', bins=20, ax=ax2
    )
    df_estadisticas[df_estadisticas['dentro_fuera'] == 'externo']['media_tc'].hist(
        alpha=0.7, label='Externo', bins=20, ax=ax2
    )
    ax2.set_title('Distribución de Cobertura Forestal')
    ax2.set_xlabel('% Cobertura Arbórea')
    ax2.legend()

    # Gráfico 3: Regresión
    ax3 = axes[1, 0]
    interno = df_regresion[df_regresion['D'] == 1]['media_tc']
    externo = df_regresion[df_regresion['D'] == 0]['media_tc']

    ax3.scatter([0] * len(externo), externo, alpha=0.6, label='Externo', s=30)
    ax3.scatter([1] * len(interno), interno, alpha=0.6, label='Interno', s=30)

    # Línea de regresión
    medias = [externo.mean(), interno.mean()]
    ax3.plot([0, 1], medias, 'r-', linewidth=2, label='Regresión')

    ax3.set_

INTENTO DEL CREDITO EXTRA


In [ ]:
# Análisis de Regresión Discontinua (RD) a Nivel de Píxel
#CREDITO EXTRA
import geopandas as gpd
import rasterio
import numpy as np
import pandas as pd
from rasterio.features import rasterize
from rasterio.transform import rowcol, xy
from scipy.spatial.distance import cdist
from scipy.ndimage import distance_transform_edt
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
import seaborn as sns
from shapely.geometry import Point
import warnings
warnings.filterwarnings('ignore')


# PREPARO DATOS Y MÁSCARAS

def crear_datos_ejemplo():
    """
    Crea datos de ejemplo para demostrar el análisis RD
    """
    print(" Creando datos de ejemplo...")

    # Creo área protegida de ejemplo
    from shapely.geometry import Polygon

    # Área protegida rectangular
    area_protegida = Polygon([
        (-75.2, -10.2),
        (-74.8, -10.2),
        (-74.8, -9.8),
        (-75.2, -9.8),
        (-75.2, -10.2)
    ])

    gdf = gpd.GeoDataFrame([{'id': 'PA_1', 'geometry': area_protegida}], crs='EPSG:4326')

    # Creo ráster de cobertura forestal simulado
    height, width = 400, 400

    # Coordenadas del ráster
    bounds = (-75.4, -10.4, -74.6, -9.6)
    transform = rasterio.transform.from_bounds(*bounds, width, height)

    # Simulo cobertura forestal con efecto de borde
    np.random.seed(42)
    x = np.linspace(-75.4, -74.6, width)
    y = np.linspace(-10.4, -9.6, height)
    X, Y = np.meshgrid(x, y)

    # Creo efecto de gradiente hacia el área protegida
    center_x, center_y = -75.0, -10.0
    distance_to_center = np.sqrt((X - center_x)**2 + (Y - center_y)**2)

    # Cobertura base con efecto espacial
    base_coverage = 60 + 30 * np.exp(-distance_to_center * 50)

    # Agrego ruido
    noise = np.random.normal(0, 10, (height, width))
    forest_coverage = np.clip(base_coverage + noise, 0, 100)

    profile = {
        'driver': 'GTiff',
        'dtype': 'float32',
        'nodata': -9999,
        'width': width,
        'height': height,
        'count': 1,
        'crs': 'EPSG:4326',
        'transform': transform
    }

    return gdf, forest_coverage.astype('float32'), profile

def cargar_datos_reales(shapefile_path, raster_path):
    """
    Carga datos reales si están disponibles
    """
    try:
        gdf = gpd.read_file(shapefile_path)
        with rasterio.open(raster_path) as src:
            forest_data = src.read(1)
            profile = src.profile
        print(" Datos reales cargados")
        return gdf, forest_data, profile
    except:
        print(" Usando datos de ejemplo")
        return crear_datos_ejemplo()


# CALCULO DISTANCIA AL LÍMITE

def crear_mascara_area_protegida(gdf, raster_profile):
    """
    Crea máscara binaria del área protegida en el ráster
    """
    print(" Creando máscara de área protegida...")

    # Rasterizo el polígono
    shapes = [(geom, 1) for geom in gdf.geometry]

    mask = rasterize(
        shapes=shapes,
        out_shape=(raster_profile['height'], raster_profile['width']),
        transform=raster_profile['transform'],
        fill=0,
        dtype='uint8'
    )

    return mask.astype(bool)

def calcular_distancia_al_limite(mask, transform):
    """
    Calcula la distancia de cada píxel al límite del área protegida

    Args:
        mask: Máscara binaria (True = dentro, False = fuera)
        transform: Transformación georreferenciada del ráster

    Returns:
        distance_array: Array con distancias (+ = interno, - = externo)
    """
    print(" Calculando distancias al límite...")

    # Distancia euclidiana desde el interior hacia el borde
    dist_interior = distance_transform_edt(mask)

    # Distancia euclidiana desde el exterior hacia el borde
    dist_exterior = distance_transform_edt(~mask)

    # Combino: positivo = interno, negativo = externo
    distance_array = np.where(mask, dist_interior, -dist_exterior)

    # Convierto píxeles a metros (aproximación)
    pixel_size_degrees = abs(transform[0])  # Tamaño de píxel en grados
    meters_per_degree = 111320  # Aproximadamente en el ecuador
    pixel_size_meters = pixel_size_degrees * meters_per_degree

    distance_meters = distance_array * pixel_size_meters

    return distance_meters

# PREPARO LOS DATOS A NIVEL DE PÍXEL

def extraer_datos_buffer_25km(distance_array, forest_data, buffer_km=25):
    """
    Extrae datos de píxeles dentro del buffer de 25km del límite
    """
    print(f"🎯 Extrayendo píxeles dentro de {buffer_km}km del límite...")

    buffer_meters = buffer_km * 1000

    # Máscara para píxeles dentro del buffer
    buffer_mask = np.abs(distance_array) <= buffer_meters

    # Extraigo datos válidos
    valid_mask = buffer_mask & (forest_data != -9999) & (~np.isnan(forest_data))

    distances = distance_array[valid_mask]
    forest_values = forest_data[valid_mask]

    # Creo DataFrame
    df_pixels = pd.DataFrame({
        'distance_m': distances,
        'forest_cover': forest_values,
        'inside': distances >= 0
    })

    print(f"  Extraídos {len(df_pixels):,} píxeles válidos")
    print(f"   - Internos: {(df_pixels['inside']).sum():,}")
    print(f"   - Externos: {(~df_pixels['inside']).sum():,}")

    return df_pixels

# HAGO EL ANÁLISIS DE REGRESIÓN DISCONTINUA

def ejecutar_rd_analysis(df_pixels, bandwidth_m=5000, poly_degree=2):
    """
    Ejecuta análisis de Regresión Discontinua

    Args:
        df_pixels: DataFrame con datos de píxeles
        bandwidth_m: Ancho de banda alrededor del límite (metros)
        poly_degree: Grado del polinomio para controles flexibles
    """
    print(f"\n Ejecutando Regresión Discontinua...")
    print(f"   - Ancho de banda: ±{bandwidth_m/1000:.1f}km")
    print(f"   - Grado polinomial: {poly_degree}")

    # Filtro datos dentro del ancho de banda
    df_rd = df_pixels[np.abs(df_pixels['distance_m']) <= bandwidth_m].copy()

    if len(df_rd) == 0:
        print(" No hay datos dentro del ancho de banda")
        return None

    # Normalizo las distancia (dividir por 1000 para trabajar en km)
    df_rd['distance_km'] = df_rd['distance_m'] / 1000
    df_rd['treatment'] = df_rd['inside'].astype(int)

    # Centrar distancia en el límite
    df_rd['distance_centered'] = df_rd['distance_km']

    # Creo variables polinomiales
    X_vars = []

    # Variable de tratamiento (discontinuidad)
    X_vars.append(df_rd['treatment'].values.reshape(-1, 1))

    # Controles polinomiales de distancia
    for degree in range(1, poly_degree + 1):
        X_vars.append((df_rd['distance_centered'] ** degree).values.reshape(-1, 1))

    # Interacciones tratamiento × distancia
    for degree in range(1, poly_degree + 1):
        interaction = (df_rd['treatment'] * (df_rd['distance_centered'] ** degree)).values.reshape(-1, 1)
        X_vars.append(interaction)

    # Combino todas las variables
    X = np.hstack(X_vars)
    y = df_rd['forest_cover'].values

    # Regresión
    model = LinearRegression()
    model.fit(X, y)

    # Predicciones
    y_pred = model.predict(X)

    # El coeficiente de tratamiento es el primero
    rd_effect = model.coef_[0]
    intercept = model.intercept_

    # R²
    ss_res = np.sum((y - y_pred) ** 2)
    ss_tot = np.sum((y - np.mean(y)) ** 2)
    r2 = 1 - (ss_res / ss_tot)

    # Estadísticas descriptivas
    stats = {
        'n_total': len(df_rd),
        'n_inside': df_rd['treatment'].sum(),
        'n_outside': len(df_rd) - df_rd['treatment'].sum(),
        'mean_inside': df_rd[df_rd['inside']]['forest_cover'].mean(),
        'mean_outside': df_rd[~df_rd['inside']]['forest_cover'].mean(),
        'rd_effect': rd_effect,
        'intercept': intercept,
        'r2': r2,
        'bandwidth_km': bandwidth_m / 1000
    }

    print(f"\n RESULTADOS DE REGRESIÓN DISCONTINUA:")
    print(f"   • Observaciones: {stats['n_total']:,}")
    print(f"   • Efecto RD (β₁): {rd_effect:.3f} puntos porcentuales")
    print(f"   • Cobertura en el límite (exterior): {intercept:.2f}%")
    print(f"   • R²: {r2:.3f}")
    print(f"   • Media interna: {stats['mean_inside']:.2f}%")
    print(f"   • Media externa: {stats['mean_outside']:.2f}%")

    return {
        'model': model,
        'data': df_rd,
        'stats': stats,
        'X': X,
        'y': y,
        'y_pred': y_pred
    }

# LUEGO LAS VISUALIZACIONES

def crear_graficos_rd(rd_results, df_pixels):
    """
    Crea gráficos de Regresión Discontinua
    """
    print("\n Generando gráficos...")

    fig, axes = plt.subplots(2, 2, figsize=(16, 12))

    df_rd = rd_results['data']
    stats = rd_results['stats']

    # Gráfico 1: RD Plot Principal
    ax1 = axes[0, 0]

    # Datos agrupados en bins para visualización
    bins_inside = np.linspace(0, stats['bandwidth_km'], 20)
    bins_outside = np.linspace(-stats['bandwidth_km'], 0, 20)

    # Calculo medias por bin
    def bin_means(distances, values, bins):
        bin_centers = []
        bin_means_vals = []
        for i in range(len(bins)-1):
            mask = (distances >= bins[i]) & (distances < bins[i+1])
            if mask.sum() > 0:
                bin_centers.append((bins[i] + bins[i+1]) / 2)
                bin_means_vals.append(values[mask].mean())
        return np.array(bin_centers), np.array(bin_means_vals)

    # Datos internos
    inside_data = df_rd[df_rd['inside']]
    if len(inside_data) > 0:
        bin_centers_in, bin_means_in = bin_means(
            inside_data['distance_km'], inside_data['forest_cover'], bins_inside
        )
        ax1.scatter(bin_centers_in, bin_means_in, c='green', s=50, alpha=0.7, label='Interno')

    # Datos externos
    outside_data = df_rd[~df_rd['inside']]
    if len(outside_data) > 0:
        bin_centers_out, bin_means_out = bin_means(
            outside_data['distance_km'], outside_data['forest_cover'], bins_outside
        )
        ax1.scatter(bin_centers_out, bin_means_out, c='red', s=50, alpha=0.7, label='Externo')

    # Línea de regresión
    x_smooth = np.linspace(-stats['bandwidth_km'], stats['bandwidth_km'], 100)

    # Predicción para visualización (simplificada)
    y_left = stats['intercept'] + 0 * x_smooth[x_smooth < 0]  # Lado externo
    y_right = stats['intercept'] + stats['rd_effect'] + 0 * x_smooth[x_smooth >= 0]  # Lado interno

    if len(x_smooth[x_smooth < 0]) > 0:
        ax1.plot(x_smooth[x_smooth < 0], y_left, 'r-', linewidth=2, alpha=0.8)
    if len(x_smooth[x_smooth >= 0]) > 0:
        ax1.plot(x_smooth[x_smooth >= 0], y_right, 'g-', linewidth=2, alpha=0.8)

    # Línea vertical en el límite
    ax1.axvline(x=0, color='black', linestyle='--', alpha=0.5, label='Límite AP')

    ax1.set_xlabel('Distancia al Límite (km)')
    ax1.set_ylabel('Cobertura Forestal (%)')
    ax1.set_title(f'Regresión Discontinua\nEfecto: {stats["rd_effect"]:.2f} pp')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Gráfico 2: Distribución de distancias
    ax2 = axes[0, 1]
    df_rd['distance_km'].hist(bins=30, alpha=0.7, ax=ax2)
    ax2.axvline(x=0, color='red', linestyle='--', label='Límite')
    ax2.set_xlabel('Distancia al Límite (km)')
    ax2.set_ylabel('Número de Píxeles')
    ax2.set_title('Distribución de Distancias')
    ax2.legend()

    # Gráfico 3: Cobertura vs Distancia (scatter completo)
    ax3 = axes[1, 0]

    # Submuestreo para visualización si hay muchos puntos
    if len(df_rd) > 5000:
        df_sample = df_rd.sample(5000, random_state=42)
    else:
        df_sample = df_rd

    inside_sample = df_sample[df_sample['inside']]
    outside_sample = df_sample[~df_sample['inside']]

    ax3.scatter(outside_sample['distance_km'], outside_sample['forest_cover'],
               c='red', alpha=0.3, s=1, label='Externo')
    ax3.scatter(inside_sample['distance_km'], inside_sample['forest_cover'],
               c='green', alpha=0.3, s=1, label='Interno')

    ax3.axvline(x=0, color='black', linestyle='--', alpha=0.5)
    ax3.set_xlabel('Distancia al Límite (km)')
    ax3.set_ylabel('Cobertura Forestal (%)')
    ax3.set_title('Datos Individuales por Píxel')
    ax3.legend()

    # Gráfico 4: Comparación de medias
    ax4 = axes[1, 1]

    means = [stats['mean_outside'], stats['mean_inside']]
    labels = ['Externo', 'Interno']
    colors = ['red', 'green']

    bars = ax4.bar(labels, means, color=colors, alpha=0.7)
    ax4.set_ylabel('Cobertura Forestal Promedio (%)')
    ax4.set_title('Comparación de Medias')

    # Agrego valores en las barras
    for bar, mean in zip(bars, means):
        height = bar.get_height()
        ax4.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                f'{mean:.1f}%', ha='center', va='bottom')

    # Línea de diferencia
    ax4.annotate('', xy=(0.1, stats['mean_outside']), xytext=(0.9, stats['mean_inside']),
                arrowprops=dict(arrowstyle='<->', color='blue', lw=2))
    ax4.text(0.5, (stats['mean_inside'] + stats['mean_outside'])/2 + 1,
             f'Δ = {stats["rd_effect"]:.2f} pp', ha='center', color='blue', fontweight='bold')

    plt.tight_layout()
    plt.savefig('rd_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()

# LA FUNCIÓN PRINCIPAL

def analisis_rd_completo(shapefile_path=None, raster_path=None):
    """
    Ejecuta el análisis completo de Regresión Discontinua
    """
    print(" ANÁLISIS DE REGRESIÓN DISCONTINUA - NIVEL DE PÍXEL")
    print("=" * 60)

    # 1. Cargo datos
    if shapefile_path and raster_path:
        gdf, forest_data, profile = cargar_datos_reales(shapefile_path, raster_path)
    else:
        gdf, forest_data, profile = crear_datos_ejemplo()

    # 2. Creo máscara de área protegida
    mask = crear_mascara_area_protegida(gdf, profile)

    # 3. Calculo distancias al límite
    distance_array = calcular_distancia_al_limite(mask, profile['transform'])

    # 4. Extraigo datos en buffer de 25km
    df_pixels = extraer_datos_buffer_25km(distance_array, forest_data, buffer_km=25)

    # 5. Análisis RD con diferentes anchos de banda
    print(f"\n🔬 PROBANDO DIFERENTES ANCHOS DE BANDA:")

    bandwidths = [2, 5, 10, 15]  # km
    rd_results_list = []

    for bw_km in bandwidths:
        print(f"\n--- Ancho de banda: {bw_km}km ---")
        rd_result = ejecutar_rd_analysis(df_pixels, bandwidth_m=bw_km*1000, poly_degree=2)
        if rd_result:
            rd_results_list.append({
                'bandwidth': bw_km,
                'effect': rd_result['stats']['rd_effect'],
                'r2': rd_result['stats']['r2'],
                'n_obs': rd_result['stats']['n_total']
            })

    # 6. Uso el ancho de banda óptimo (5km por defecto)
    rd_final = ejecutar_rd_analysis(df_pixels, bandwidth_m=5000, poly_degree=2)

    # 7. Creo visualizaciones
    if rd_final:
        crear_graficos_rd(rd_final, df_pixels)

    # 8. Resumen de resultados
    print(f"\n RESUMEN DE RESULTADOS:")
    print("-" * 40)
    for result in rd_results_list:
        print(f"BW={result['bandwidth']:2d}km: Efecto={result['effect']:+6.2f}pp, "
              f"R²={result['r2']:.3f}, N={result['n_obs']:,}")

    return {
        'distance_array': distance_array,
        'df_pixels': df_pixels,
        'rd_results': rd_final,
        'sensitivity': rd_results_list,
        'mask': mask
    }

# EJECUTO EL ANÁLISIS

if __name__ == "__main__":
    # Ejecutar con datos de ejemplo
    resultados = analisis_rd_completo()

    # Para usar tus propios datos, descomenta y ajusta:
    # resultados = analisis_rd_completo(
    #     shapefile_path="areas_protegidas.shp",
    #     raster_path="cobertura_forestal.tif"
    # )